In [ ]:
%reload_ext eradiate.notebook
import matplotlib.pyplot as plt
import mitsuba as mi

import eradiate
from eradiate import KernelContext
from eradiate import unit_registry as ureg
from eradiate.kernel import scene_parameter
from eradiate.xarray.interp import dataarray_to_rgb

eradiate.set_mode("mono")

In [ ]:
@scene_parameter
def spectrum_lambertian(ctx: KernelContext):
    if ctx.si.w == 440.0 * ureg.nm:
        return 0.1
    elif ctx.si.w == 550.0 * ureg.nm:
        return 0.5
    elif ctx.si.w == 660.0 * ureg.nm:
        return 0.9
    else:
        return 0.0


display(spectrum_lambertian)

exp = eradiate.experiments.AtmosphereExperiment(
    atmosphere={"type": "molecular"},
    surface={"type": "lambertian"},
    measures={
        "type": "perspective",
        "origin": [5, 5, 1],
        "target": [0, 0, 0.5],
        "film_resolution": (320, 240),
        "srf": {"type": "delta", "wavelengths": [440, 550, 660]},
    },
    kdict={
        "mat_laboule": {"type": "diffuse", "id": "mat_laboule"},
        "mat_lecarre": {"type": "roughplastic", "id": "mat_lecarre"},
        "sphere": {
            "type": "sphere",
            "center": [0, 0, 0],
            "id": "laboule",
            "bsdf": {"type": "ref", "id": "mat_laboule"},
        },
        "square": {
            "type": "rectangle",
            "to_world": mi.ScalarTransform4f.translate([0, 0, 0.1])
            @ mi.ScalarTransform4f.scale(2.0),
            "bsdf": {"type": "ref", "id": "mat_lecarre"},
        },
    },
    kpmap={
        "mat_laboule.reflectance.value": spectrum_lambertian,
    },
)
exp.init()
exp.mi_scene.parameters

In [ ]:
result = eradiate.run(exp, spp=256)
result.radiance.plot(yincrease=False, aspect=5 / 3, size=4)

In [ ]:
img = dataarray_to_rgb(result.radiance, channels=[("w", 660), ("w", 550), ("w", 440)])
plt.axis("off")
plt.imshow(img)
plt.savefig("expert.png", bbox_inches="tight", dpi=300)